# Content-based Filtering

Aproach:
- Table with features erstellen, ratings in zweitem table

- sklearn-kNN mit minkowski-distance auf features anwenden, erhalte matrix mit allen Nachbarn eines jeden Films inklusive Abstand

- Wähle für jeden Nutzer und Film die x-nächsten Nachbarn aus, die der Nutzer bewertet hat

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

In [ ]:
import pandas as pd
import numpy as np
import string
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
import time
kwargs = dict(random_state=42)

In [ ]:
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

In [ ]:
omdb.columns

Prepare data

In [ ]:
omdb = omdb.reset_index()
omdb_index = omdb[['index', '0']]

In [ ]:
ratings = ratings.merge(omdb_index, left_on= 'imdbID', right_on='0', how='left').drop(['0'], axis=1)
#delete ratings of movie "nomads"
ratings = ratings[ratings['imdbID']!='tt0720339']

Seleczt

In [ ]:
import knn_Carmen

In [ ]:
def grid_search_kNN(n_samples, params_feature):
    features, names = knn_Carmen.features(**params_feature)
    zeit_1 = time.perf_counter()
    distances, indices = knn_Carmen.calculate_KNN(features)
    zeit_knn = time.perf_counter() - zeit_1
    print('Gesamtzeit_KNN: '+str(zeit_knn))

    test_set = ratings.sample(n_samples)
    predictions = np.empty(n_samples)
    reals = np.empty(n_samples)
    i = 0
    zeit_2 = time.perf_counter()
    for index, row in test_set.iterrows():
            # select imdbID, userID and real rating
            imdbID = row['imdbID']
            userID = row['user_id']
            real = row['rating']

            # Index of movie in omdb dataframe
            index = omdb.loc[(omdb['imdbID'] == imdbID)].index
            # Store order of indices of the neighbors and tausche erste Reihe mit Spaltenbezeichnungen
            movie_row = indices.loc[index]
            # Select the movie in the distance matrix
            movie_row_distance = distances.loc[index]
            # Number of existing neighbors
            n_neighbors = len(indices)
            # ratings of the user    
            rated_id = ratings.loc[ratings['user_id'] == userID]
            # predict rating
            pred = knn_Carmen.predict_rating(index, movie_row, movie_row_distance, indices, n_neighbors, rated_id, imdbID, userID)
            predictions[i] = pred
            reals[i] = real
            i = i+1
    rmse = mean_squared_error(reals, predictions, squared=False)
    zeit_samples = time.perf_counter() - zeit_2
    print('RMSE: '+str(rmse))
    print('Gesamtzeit_Samples: '+str(zeit_samples))

    results = pd.DataFrame()
    results['Features'] = [names]
    results['Sample_size'] = n_samples
    results['RMSE'] = rmse
    results['Runtime_total'] = zeit_knn+zeit_samples
    results['Runtime_KNN'] = zeit_knn
    results['Runtime_samples'] = zeit_samples
    
    try:
        results_total = pd.read_csv('Ergebnisse_Carmen_ContentBased.csv')
        results_total = results_total.append(results)
    except FileNotFoundError:
        results_total = results
    results_total[['Features', 'Sample_size', 'RMSE', 'Runtime_total', 'Runtime_KNN', 'Runtime_samples']].to_csv('Ergebnisse_Carmen_ContentBased.csv')

In [ ]:
#params = {'threshold_actors': [0, 20, 40], 'ts_languages': [10, 20] , 'year': [True, False], 'runtime': [True, False], 'imdbVotes': [True, False], 'series': [True, False], 'awards': [True, False], 'genres': [True, False]}

In [ ]:
params = {'threshold_actors': [50], 'ts_languages': [20] , 'year': [True], 'runtime': [False], 'imdbVotes': [False], 'series': [False], 'awards': [False], 'genres': [True], 'rating':[True], 'rated':[True]}

In [ ]:
from itertools import product
permutations = [dict(zip(params, v)) for v in product(*params.values())]

In [ ]:
permutations

In [ ]:
n_samples = 10000
for i in permutations:
    grid_search_kNN(n_samples, i)

In [ ]:
pd.read_csv('Ergebnisse_Carmen_ContentBased.csv').columns

Backup

In [ ]:
# Use optimal k
def adjust_k(r_size):
    adjusted_k = 10
    if r_size > 40 and r_size  < 100:
        adjusted_k = 15
    elif r_size  > 100 and r_size < 500:
        adjusted_k = 20
    elif r_size  > 500 and r_size < 1500:
        adjusted_k = 25
    elif r_size  > 1500:
        adjusted_k = 30
        
    return adjusted_k

In [ ]:
#Alles in eine Funktion
def predict_rating(imdbID, userID, mean=True):
    zeitu = time.perf_counter()
    
    #Bestimme Index in omdb von der ImdbID
    index = omdb.loc[(omdb['imdbID'] == imdbID)].index
    
    #Speichere Reihenfolge der Indizes der Nachbarn des Films und tausche erste Reihe mit Spaltenbezeichnungen
    movie_row = indices_df.loc[index]
    movie_row = movie_row.reset_index(drop=True)
    movie_row.columns = movie_row.loc[0]
    movie_row.drop(0, inplace=True)
    movie_row.loc[1,:] = range(0,len(indices_df))
    
    #Speichere alle Filme, die der User bewertet hat und bestimme index davon
    rated_id = ratings.loc[ratings['user_id'] == userID]
    rated_df = rated_id['index']
    rated_df = pd.DataFrame(rated_df)
    
    # Bestimme die Position der bewerteten Filme innerhalb der Nachbarschaft
    position = movie_row[list(rated_df['index'].astype(int))]
    position = position.transpose()
    position.columns = ['position']
    position ['index'] = position.index
    position = position.reset_index(drop = True)
    
    #Greife auf den Film in der Distanzmatrix zu
    movie_row_distance = distances_df.loc[index]
    
    #Bestimme Distanz der Filme vom Film, die der User bewertet hat
    user_distances = movie_row_distance[list(position['position'].astype(int))].transpose()
    user_distances.columns = ['distance']
    user_distances =user_distances.reset_index(drop = True)
    
    #Füge alle Infos zusammen und sortiere von nächstem zum weitest entfernten Film
    neighbors = pd.concat([position, user_distances], axis=1, join = "inner")
    neighbors = neighbors.sort_values("distance")
        
    #Füge die Ratings des Users zu diesen Filmen noch hinzu
    neighbors = neighbors.merge(rated_id[["rating","index"]], on="index", how = "outer")
    neighbors = neighbors.iloc[1:]
    
    #Berechne die Prognose: 
    if mean==True:
        # Mittelwer der k-nächsten Nachbarn
        k = adjust_k(neighbors.shape[0])
        pred = neighbors['rating'].iloc[:k].mean()
    else:
        # Gewichtung der Bewertung der Nachbarn je nach Distanz
        pred = sum(neighbors['rating']*((1/neighbors['distance'])**10)/(sum((1/neighbors['distance'])**10)))
    
    return pred

In [ ]:
n = 100
test_set = ratings.sample(n)
test_set['rating'].isna().sum()
test_set